In [121]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from datetime import timedelta

# Cargar datos desde un archivo CSV (ajusta la ruta según sea necesario)
df = pd.read_csv('/kaggle/input/kdp3g24/KDP6.csv')

# Asegurar que 'Fecha de pedido' sea datetime
df['Fecha de pedido'] = pd.to_datetime(df['Fecha de pedido'], errors='coerce')

# Asegurar que 'Regalias' sea numérica (eliminar caracteres no numéricos)
df['Regalias'] = pd.to_numeric(df['Regalias'].str.replace(',', ''), errors='coerce')

# Guardar una correspondencia entre 'ID' y 'Titulo_del_Libro' antes de eliminar duplicados
titulo_correspondencia = df[['ID', 'Titulo_del Libro']].drop_duplicates()

# Asegurar que 'Titulo_del_Libro' esté presente antes de eliminar filas con valores faltantes
df = df.dropna(subset=['Titulo_del Libro', 'Regalias', 'Fecha de pedido'])

# Eliminar 'Titulo_del_Libro' del DataFrame antes de codificar 'ID'
df = df.drop('Titulo_del Libro', axis=1)

# Codificar 'ID' usando LabelEncoder después de guardar la correspondencia
le = LabelEncoder()
df['ID'] = le.fit_transform(df['ID'])

# Asegurar que 'ID' sea int
df['ID'] = df['ID'].astype(int)

# Convertir 'Fecha de pedido' a números enteros (UNIX timestamp)
df['Fecha de pedido'] = df['Fecha de pedido'].astype(int) // 10**9

# Dividir el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(df[['ID', 'Fecha de pedido']], df['Regalias'], test_size=0.2, random_state=42)

# Restablecer el índice después de la división
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

# Proporcionar nombres de características al modelo
X_train.columns = ['ID', 'Fecha de pedido']

# Inicializar el modelo
modelo = LinearRegression()

# Entrenar el modelo
modelo.fit(X_train, y_train)


# Crear un DataFrame para almacenar las predicciones futuras
predicciones_futuras = pd.DataFrame(columns=['ID', 'Titulo_del Libro', 'Fecha de pedido', 'Prediccion Regalias'])

# Generar predicciones para cada libro
for ID in df['ID'].unique():
    # Filtrar datos para el libro específico
    datos_libro = df[df['ID'] == ID]

    # Obtener la fecha máxima presente en el conjunto de datos para ese libro
    fecha_maxima = pd.to_datetime(datos_libro['Fecha de pedido'].max())  # Convertir de nuevo a datetime

    # Calcular fecha futura
    fecha_futura = fecha_maxima + timedelta(days=30)  # Ajusta el número de días según sea necesario

    # Convertir la fecha a un número
    fecha_futura_numerica = fecha_futura.timestamp()

    # Realizar la predicción
    prediccion = modelo.predict([[ID, fecha_futura_numerica]])  # Se pasa tanto 'ID' como 'Fecha de pedido'

    # Verificar si hay alguna fila correspondiente a ID en titulo_correspondencia
    if ID in titulo_correspondencia['ID'].values:
        # Obtener el título correspondiente al 'ID'
        titulo_correspondiente = titulo_correspondencia[titulo_correspondencia['ID'] == ID]['Titulo_del Libro'].iloc[0]

        # Agregar la predicción al DataFrame
        predicciones_futuras = pd.concat([predicciones_futuras, pd.DataFrame({
            'ID': [ID],
            'Titulo_del_Libro': [titulo_correspondiente],
            'Fecha de pedido': [fecha_futura],
            'Prediccion Regalias': [prediccion[0]]
        })], ignore_index=True)
    else:
        print(f"No se encontró ninguna correspondencia para el ID {ID}.")

# Mostrar el DataFrame con las predicciones en euros
predicciones_futuras['Regalias en euros'] = predicciones_futuras['Prediccion Regalias'] * 0.85  # Assuming 1 USD = 0.85 EUR
print(predicciones_futuras)

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature n

No se encontró ninguna correspondencia para el ID 0.
      ID Titulo_del Libro               Fecha de pedido  Prediccion Regalias  \
0    210              NaN 1970-01-31 00:00:01.704067200         32964.736583   
1    209              NaN 1970-01-31 00:00:01.704067200         32962.263507   
2    208              NaN 1970-01-31 00:00:01.701388800         32959.790432   
3    207              NaN 1970-01-31 00:00:01.701388800         32957.317357   
4    206              NaN 1970-01-31 00:00:01.701388800         32954.844281   
..   ...              ...                           ...                  ...   
205    5              NaN 1970-01-31 00:00:01.654041600         32457.756125   
206    4              NaN 1970-01-31 00:00:01.654041600         32455.283050   
207    3              NaN 1970-01-31 00:00:01.651363200         32452.809974   
208    2              NaN 1970-01-31 00:00:01.651363200         32450.336899   
209    1              NaN 1970-01-31 00:00:01.651363200         324

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature n